In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "codellama/CodeLlama-34b-hf"

peft_model_id = "/home/ksaff/Desktop/ttyd/fine_tuning/2nd_try"
config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16
                                )

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config, device_map={'': 0}
)
tokenizer = AutoTokenizer.from_pretrained(
                                          config.base_model_name_or_path)

/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model.safetensors.index.json
['/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00001-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00002-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00003-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00004-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00005-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--Cod

Loading checkpoint shards: 100%|██████████| 7/7 [00:18<00:00,  2.67s/it]


In [34]:
from transformers import pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
hf = HuggingFacePipeline(pipeline=pipe)

In [33]:
memory = ConversationBufferMemory()
conversation_buf = ConversationChain(
    llm=hf,
    memory=memory)

ValidationError: 1 validation error for ConversationChain
llm
  Can't instantiate abstract class BaseLanguageModel with abstract methods agenerate_prompt, apredict, apredict_messages, generate_prompt, invoke, predict, predict_messages (type=type_error)

In [ ]:
database_scheme = """
CREATE TABLE Salaries
  Id INTEGER PRIMARY KEY, -- Unique ID for each employee
  EmployeeName VARCHAR, -- Name of the employee
  JobTitle VARCHAR, -- Name of employees proffesion
  BasePay NUMERIC, -- Base pay of employee
  OvertimePay NUMERIC, -- Overtime pay of employee
  OtherPay NUMERIC, -- Other pays of employee
  Benefits NUMERIC, -- Benefits of employee
  TotalPay NUMERIC, -- Total pay of employee
  TotalPayBenefits NUMERIC, -- Sum of pay benefits of employee
  Year INTEGER, -- Year data from row reffers to
"""

question = "Calculate the Total Pay Including Benefits for Each Employee"

text = (
    """### Task
    Generate a SQL query to answer the following question:
    {question}
### Database Schema
    This query will run on a database whose schema is represented in this string:
    {database_scheme}
### SQL
    Given the database schema, here is the SQL query that answers `{question}`:
    ```sql
    """.format(question=question, database_scheme=database_scheme)
)